In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from tqdm.keras import TqdmCallback
from PIL import Image
import glob

Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

"""SAMPLE: Image background manipulation. A starting point of RGB manibulations.
Under certain conditions, it will be possible to remove most artifacts and unwanted images that aren't relevant.
Method 1: Transparent background
Method 2: Testing RGB sets to determine the one improving on French Fries black spots indentification.
In this particular test, the inverse of the RGB set of the black spot region has been the most effective
"""
def convertImage():
    img = Image.open("fry_set_train1.jpg")
    img = img.convert("RGBA")
  
    datas = img.getdata()
  
    newData = []
  
    for item in datas:
        if item[0] < 100 and item[1] < 100 and item[2] < 100:
            newData.append((0, 0, 0, 0))
        else:
            newData.append(item)
  
    img.putdata(newData)
    img.save("fry_set_train1_bkg.png")
    print("Successful generating: fry_set_train1_bkg.png")

In [3]:
"""Test wrapper"""
convertImage()

"""A better background of the trainning data image"""
im2 = Image.open("fry_set_train1_bkg.png")
im2.show()

Successful generating: fry_set_train1_bkg.png


In [4]:
raw_image_list = []
for filename in glob.glob("C:/Users/seeho/datasets/fries/*.jpg"): #change your path here!!!
    im=Image.open(filename)
    im.show()
    raw_image_list.append(im)
 
"""Did we get the content here?"""
l=len(raw_image_list)
print(l)

3


In [5]:
"""
Note: The issue is that Jpegs don't have a shape. Instead they use img.size.
Pick one random image from the list"""
x = raw_image_list[2]
h, w = x.size
print("original shape", x.size)
print('width:  ', w)
print('height: ', h)

new_width = 150;
new_height =150;

img = x.resize((new_width, new_height), Image.ANTIALIAS);

x = img_to_array(img)

print("Modified shape step 1", x.shape)

x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
print("Modified shape step 2", x.shape)

original shape (3356, 2237)
width:   2237
height:  3356
Modified shape step 1 (150, 150, 3)
Modified shape step 2 (1, 150, 150, 3)


In [6]:
# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `train/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='C:/Users/seeho/data/train/fries', save_prefix='train', save_format='jpeg'):
    i += 1
    if i > 50:
        break  # otherwise the generator would loop indefinitely
        
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='C:/Users/seeho/data/validation/fries', save_prefix='val', save_format='jpeg'):
    i += 1
    if i > 50:
        break  # otherwise the generator would loop indefinitely

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


"""Dimensions of our images that matches the previous resizing step"""
img_width, img_height = 150, 150

train_data_dir = 'C:/Users/seeho/data/train/fries'
validation_data_dir = 'C:/Users/seeho/data/validation/fries'
test_data_item = 'C:/Users/seeho/data/test/fries'

nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)       

In [9]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1./255)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        directory='C:/Users/seeho/data/train/',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        directory='C:/Users/seeho/data/validation/',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 204 images belonging to 2 classes.
Found 205 images belonging to 2 classes.


In [10]:
model.fit_generator(
        train_generator,
        steps_per_epoch=100 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=100 // batch_size,
        verbose=2, callbacks=[TqdmCallback(verbose=2)])

# always save your weights after training or during training
model.save('first_try.h5', overwrite=True)  
print("Saved model first_try.h5 on the project directory")

Epoch 1/10


 - 3s - loss: 0.7226 - accuracy: 0.6562 - val_loss: 0.1573 - val_accuracy: 0.9896
Epoch 2/10



 - 2s - loss: 0.0918 - accuracy: 0.9674 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 3/10



 - 2s - loss: 0.0042 - accuracy: 1.0000 - val_loss: 1.0716e-04 - val_accuracy: 1.0000
Epoch 4/10



 - 2s - loss: 6.8937e-04 - accuracy: 1.0000 - val_loss: 1.4729e-05 - val_accuracy: 0.9896
Epoch 5/10



...

 - 2s - loss: 9.1900e-05 - accuracy: 1.0000 - val_loss: 1.8251e-06 - val_accuracy: 0.9896
Epoch 7/10



 - 2s - loss: 5.4430e-04 - accuracy: 1.0000 - val_loss: 8.4622e-07 - val_accuracy: 1.0000
Epoch 8/10



 - 2s - loss: 5.3629e-04 - accuracy: 1.0000 - val_loss: 1.2585e-08 - val_accuracy: 0.9896
Epoch 9/10



 - 2s - loss: 1.5207e-05 - accuracy: 1.0000 - val_loss: 2.2442e-07 - val_accuracy: 1.0000
Epoch 10/10



 - 2s - loss: 3.6708e-05 - accuracy: 1.0000 - val_loss: 5.1585e-09 - val_accuracy: 0.9896


Saved model first_try.h5 on the project directory


In [12]:
from keras.models import load_model
loaded_model=load_model("first_try.h5")

# summarize model.
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)       

In [18]:
"""This is a WRONG appoach by recycling TRAIN set to classify new image. LOL. Testing only"""
loaded_model.fit_generator(
        train_generator,
        steps_per_epoch=100 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=100 // batch_size,
        verbose=2, callbacks=[TqdmCallback(verbose=2)])

Epoch 1/10


 - 2s - loss: 1.8155e-05 - accuracy: 1.0000 - val_loss: 1.1971e-05 - val_accuracy: 0.9896
Epoch 2/10



 - 2s - loss: 1.0516e-05 - accuracy: 1.0000 - val_loss: 1.1070e-09 - val_accuracy: 1.0000
Epoch 3/10



 - 2s - loss: 1.9008e-05 - accuracy: 1.0000 - val_loss: 7.6208e-10 - val_accuracy: 0.9892
Epoch 4/10



 - 2s - loss: 6.8666e-06 - accuracy: 1.0000 - val_loss: 8.0254e-11 - val_accuracy: 1.0000
Epoch 5/10



...

 - 2s - loss: 7.7179e-05 - accuracy: 1.0000 - val_loss: 0.3626 - val_accuracy: 0.9896
Epoch 7/10



 - 2s - loss: 1.5758e-06 - accuracy: 1.0000 - val_loss: 7.4844e-13 - val_accuracy: 1.0000
Epoch 8/10



 - 2s - loss: 3.0478e-07 - accuracy: 1.0000 - val_loss: 1.0376e-12 - val_accuracy: 0.9896
Epoch 9/10



 - 2s - loss: 2.2108e-06 - accuracy: 1.0000 - val_loss: 1.7606e-09 - val_accuracy: 1.0000
Epoch 10/10



 - 2s - loss: 6.2322e-08 - accuracy: 1.0000 - val_loss: 2.2545e-13 - val_accuracy: 0.9896


